In [83]:
import sys
sys.path.append('../Scripts/')

In [84]:
import feature_engineer

### Paso 1: Inspeccionamos el Dataset, para estudiar la mejor forma de cargarlo

In [33]:
with open('../Data/dataset_SCL.csv') as file:
    n_rows = len(file.readlines())
print (f'Numero de Filas: {n_rows}')

Numero de Filas: 68207


* Con 68 mil filas el dataset facilmente cabe directamente en memoria y no requiere de procesamiento adicional, estandarizando las columnas con valores mixtos

In [34]:
data_vuelos = pd.read_csv('../Data/dataset_SCL.csv', dtype= {'Vlo-I':str, 'Vlo-O':str}, parse_dates=['Fecha-I', 'Fecha-O'])
matriz_vuelos = data_vuelos.copy()

In [35]:
matriz_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha-I    68206 non-null  datetime64[ns]
 1   Vlo-I      68206 non-null  object        
 2   Ori-I      68206 non-null  object        
 3   Des-I      68206 non-null  object        
 4   Emp-I      68206 non-null  object        
 5   Fecha-O    68206 non-null  datetime64[ns]
 6   Vlo-O      68205 non-null  object        
 7   Ori-O      68206 non-null  object        
 8   Des-O      68206 non-null  object        
 9   Emp-O      68206 non-null  object        
 10  DIA        68206 non-null  int64         
 11  MES        68206 non-null  int64         
 12  AÑO        68206 non-null  int64         
 13  DIANOM     68206 non-null  object        
 14  TIPOVUELO  68206 non-null  object        
 15  OPERA      68206 non-null  object        
 16  SIGLAORI   68206 non-null  object       

* Validacion rango de fechas vuelos

In [36]:
matriz_vuelos.aggregate({
    'Fecha-I': [min, max],
    'Fecha-O':[min,max]
})

,Fecha-I,Fecha-O
min,2017-01-01 00:15:00,2017-01-01 00:04:00
max,2017-12-31 23:55:00,2018-01-01 00:12:00


* Existe un campo vacio en Vlo-O, se revisa

In [37]:
matriz_vuelos[matriz_vuelos['Vlo-O'].isna()==True]

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,NaN,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima


* Ya que la variable es indiferente para el analisis final, no se descarta, ya que la columna misma no se usara y carece de secuencialidad.

In [38]:
t_alta1 = pd.date_range(start='01/01/2017', end='3/03/2017') 
t_alta2 = pd.date_range(start='07/15/2017', end='07/31/2017') 
t_alta3 = pd.date_range(start='9/11/2017', end='09/30/2017')
t_alta4 = pd.date_range(start='12/15/2017', end='01/01/2018')
t_alta = t_alta1.append([t_alta2, t_alta3, t_alta4])

In [78]:
test = matriz_vuelos[5:6]['Fecha-O']
test.dt.strftime('%m/%d/%Y')

5    01/07/2017
Name: Fecha-O, dtype: object

In [66]:
matriz_vuelos['dif_min']=matriz_vuelos['Fecha-O']-matriz_vuelos['Fecha-I']

In [80]:
matriz_vuelos['temporada_alta'] = pd.to_datetime(matriz_vuelos['Fecha-I']).apply(lambda x: 1 if x.strftime('%m/%d/%Y') in t_alta else 0)